# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from z3 import *
import sys
sys.path.append('../../scripts/biota/')
from BIoTA_Control import *

# Fixed Parameter

In [3]:
CO2_FRESH_AIR = 400             # CO2 concentration (ppm) of fresh air
TEMP_FRESH_AIR = 91.4           # Temperature (33 degree F) of the fresh air
CP_AIR = 1.026                  # Specific heat of fresh air
DEF_TEMP_SUPPLY_AIR =  55.4     # Default temperature (degree fahrenheit) of supply air (13 degree celsius)
MINUTES_IN_A_DAY = 1440         # Number of minutes in a day
OFF_PEAK_ENERGY_COST = 0.34     # OFF-PEAK energy cost (USD)
ON_PEAK_ENERGY_COST = 0.48      # ON-PEAK energy cost (USD)
ON_PEAK_START_SLOT = 960        # ON-PEAK start time (minute in a day)
ON_PEAK_END_SLOT = 1260         # ON-PEAK end time (minute in a day)
BATTER_STORAGE = 0.48           # Energy (kWh) produced by battery

# Variable Parameters

In [4]:
zone_temp_setpoint = [0, 75.2, 75.2, 75.2, 75.2]     # list of temperature (fahrenheit) setpoint of the different zones
zone_co2_setpoint = [0, 1000, 1000, 1000, 1000]      # list of CO2 (ppm) setpoint of the corresponding zones
control_time = 1                                     # control time (in minute)

# Accessing Zone-Activity Information

In [5]:
zones = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Zone-Info')
zone_volume = zones["Volume (cf)"].to_list()                  # Zones' volumes (cubic feet)
pp_co2 = zones["CO2 Emission by Occupant (cfm)"].to_list()    # CO2 Emission by Occupant (cfm)
pp_heat = zones["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
load = zones["Heat Radiated by Appliances (W)"].to_list()     # Heat radiated by Appliances (W)
        
activities = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Activity-Info')

activity_zone_map = dict()
for i in range(len(activities)):
    activity_zone_map[int(activities["Activity ID"][i])] = int(activities["Zone ID"][i])

# Generating Possible Combination of Occupants

In [6]:
import itertools
all_samples = list(itertools.permutations([1, 1, 0, 0, 0]))
all_samples += list(itertools.permutations([2, 0, 0, 0, 0]))
# however, there are repetations

unique_samples = set()

for data in all_samples:    
    unique_samples.add(data)
    
unique_samples

{(0, 0, 0, 0, 2),
 (0, 0, 0, 1, 1),
 (0, 0, 0, 2, 0),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 0, 2, 0, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 1, 0, 0),
 (0, 2, 0, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 1, 0, 0),
 (1, 1, 0, 0, 0),
 (2, 0, 0, 0, 0)}

# Tabulate the Control Energy Consumptions for all Possible Combinations

In [7]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_control_cost = dict()
for sample in unique_samples:
    zone_occupant = list(sample)    
    dict_control_cost[sample] = control_cost(zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time)
dict_control_cost

{(1, 0, 1, 0, 0): 0.006996611666666667,
 (1, 1, 0, 0, 0): 0.006524535000000001,
 (0, 0, 1, 0, 1): 0.036949184833333336,
 (2, 0, 0, 0, 0): 0.0,
 (0, 0, 0, 2, 0): 0.062190949333333335,
 (0, 0, 0, 0, 2): 0.03490514633333333,
 (0, 1, 0, 0, 1): 0.03647710816666667,
 (1, 0, 0, 0, 1): 0.029952573166666666,
 (0, 0, 1, 1, 0): 0.06309208633333334,
 (0, 2, 0, 0, 0): 0.012034600000000001,
 (0, 0, 0, 1, 1): 0.08604804783333334,
 (0, 1, 0, 1, 0): 0.06262000966666667,
 (0, 0, 2, 0, 0): 0.009909890000000001,
 (0, 1, 1, 0, 0): 0.013521146666666668,
 (1, 0, 0, 1, 0): 0.056095474666666666}

# BIoTA Control Cost Calculation Functions

In [8]:
def biota_control(house, filename):
    # reading processed dataset
    control_costs = []
    unit_energy_costs = []
    energy_consumptions = []
    
    dataset = pd.read_csv('../../data/processed/Processed-Dataframe_House-' + house + '_BIoTA.csv')
    
    prev_day = -1
    battery_usage = 0 # in kWh
    
    for i in range(len(dataset)):
        current_day = int(dataset.iloc[i,0])
        current_minute = int(dataset.iloc[i,1])
        
        if current_day!= prev_day:
            battery_usage = 0
            prev_day = current_day        
            print(house, current_day)
            
        zone_occupants = to_tuple(dataset.iloc[i][2:].values)
        energy_consumption = dict_control_cost[zone_occupants]
        
        if current_minute < ON_PEAK_START_SLOT or current_minute > ON_PEAK_END_SLOT:
            unit_energy_cost = OFF_PEAK_ENERGY_COST
        else:
            if battery_usage < BATTER_STORAGE:
                battery_usage += energy_consumption
                unit_energy_cost = OFF_PEAK_ENERGY_COST
            else:
                unit_energy_cost = ON_PEAK_ENERGY_COST
                
        current_cost = energy_consumption * unit_energy_cost
        control_costs.append(current_cost)
        unit_energy_costs.append(unit_energy_cost)
        energy_consumptions.append(energy_consumption)
        
    dataset['Unit Energy Cost ($)'] = unit_energy_costs
    dataset['Energy Comsumption (kWh)'] = energy_consumptions
    dataset['Control Cost ($)'] = control_costs
    
    dataset.to_csv(filename, index = False)

# Saving Datasets

In [9]:
biota_control("A", "../../data/biota/BIoTA-Control-Dataframe_House-A.csv")
biota_control("B", "../../data/biota/BIoTA-Control-Dataframe_House-B.csv")

A 1
A 2
A 3
A 4
A 5
A 6
A 7
A 8
A 9
A 10
A 11
A 12
A 13
A 14
A 15
A 16
A 17
A 18
A 19
A 20
A 21
A 22
A 23
A 24
A 25
A 26
A 27
A 28
A 29
A 30
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
B 23
B 24
B 25
B 26
B 27
B 28
B 29
B 30
